In [1]:
import os
import pandas as pd
print(f"pandas Ver.: {pd.__version__}")
import numpy as np
print(f"numpy  Ver.: {np.__version__}")
import cclib
print(f"cclib  Ver.: {cclib.__version__}")
from HessianTools import *

pandas Ver.: 2.2.3
numpy  Ver.: 1.26.4
cclib  Ver.: 1.8.1


In [2]:
# get all file names ending with .out
text_files = [
    f for f in os.listdir(os.getcwd()) 
    if f.endswith(".out")
]

In [3]:
dataframes = []
for filename in text_files:
    #load the output
    data = cclib.io.ccread(filename)

    #Create a list of 0 (carbons)
    mol_indexes = np.zeros(20)
    
    #Change the B(5) to -1 and N(7) to 1
    for i in range(20):
        if data.atomnos[i] == 5:
            mol_indexes[i] = -1
        if data.atomnos[i] == 7:
            mol_indexes[i] = 1
    
    # Create a DataFrame for the current file and add to the list
    df = pd.DataFrame([mol_indexes], columns=[f'z{i}' for i in range(20)])

    # Add the SCF Energies column
    scfEnergy = data.scfenergies
    df["Energy"] = scfEnergy

    # Add the HOMO and LUMO columns
    homo_value = data.moenergies[0][data.homos[0]]
    lumo_value = data.moenergies[0][data.homos[0] + 1]
    df["HOMO"] = homo_value
    df["LUMO"] = lumo_value

    # Load the last gradient matrix and flatten it
    grad = data.grads[-1]
    grad_flat = data.grads.flatten()

    # Create the gradient column names
    grad_column = [f"grad[{i}][{j}]" for i in range(data.natom) for j in ['x','y','z']]
    grad_df = pd.DataFrame([grad_flat], columns=grad_column)
    df = pd.concat([df, grad_df], axis=1)    

    # Load the Hessian matrix and flatten it 
    hess = HessianTools(f"{filename[:-4]}.hess")
    hessian_flat = hess.hessian.flatten()
    
    # Create Hessian column names
    hessian_column = [f"hess[{i}][{j}]" for i in range(data.natom*3) for j in range(data.natom*3)]
    hessian_df = pd.DataFrame([hessian_flat], columns=hessian_column)
    df = pd.concat([df, hessian_df], axis=1)    
    
    df.index = [filename]  # Set the filename as the index
    dataframes.append(df)
    
all_data = pd.concat(dataframes)
all_data.to_csv("output.csv", index=True)  # Use index=True to include the filename index in the CSV

In [4]:
all_data

,z0,z1,z2,z3,z4,z5,z6,z7,z8,z9,...,hess[110][101],hess[110][102],hess[110][103],hess[110][104],hess[110][105],hess[110][106],hess[110][107],hess[110][108],hess[110][109],hess[110][110]
index_246_carga_-2.out,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002526,6.012868e-08,-3.516406e-08,0.000869,-1.139343e-07,-9.413702e-09,0.004172,-3.704366e-08,-9.152461e-07,0.108446
index_248_carga_-2.out,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002246,-4.466349e-08,8.423876e-08,0.001015,-2.056329e-07,3.246595e-08,0.002489,-2.227738e-07,-1.135280e-06,0.104411
index_o_carga_0.out,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003044,1.101259e-07,-2.672121e-07,0.000773,4.335219e-08,3.088086e-07,0.003044,1.123724e-07,-9.737278e-07,0.106395
index_434_carga_0.out,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003409,-1.108170e-07,2.871318e-08,0.001108,-5.769952e-08,-1.878656e-07,0.002644,-3.952454e-08,-8.343983e-07,0.081051
index_81_carga_2.out,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.003414,3.187205e-07,-7.217261e-09,0.001556,-3.797620e-07,3.295196e-07,0.003413,1.825244e-07,-1.337350e-06,0.107217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
index_464_carga_0.out,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.003305,1.565993e-07,-4.272947e-08,0.001496,-1.567226e-07,1.186166e-07,0.002658,-1.320678e-07,-1.267071e-06,0.108931
index_55_carga_2.out,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003006,1.753479e-07,-1.486809e-08,0.000396,1.102445e-07,-7.136496e-08,0.001093,4.352615e-07,-9.963584e-07,0.100802
index_2_carga_1.out,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003329,2.140907e-11,-5.402330e-08,0.001478,-7.802823e-08,2.291803e-07,0.002986,-1.993465e-08,-1.097347e-06,0.109174
index_230_carga_-2.out,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001697,-1.251545e-07,-2.556579e-08,0.001452,5.123346e-09,1.578713e-07,0.004062,-3.904147e-08,-1.182955e-06,0.099196
